In [1]:
import numpy as np
import pandas as pd
import sys
np.set_printoptions(threshold=20)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 20)

# Input data files are available in the "../input/" directory.

import os
print(os.listdir("./data/"))

['device_data.csv', 'PAGE.csv', 'CONTENT_CATEGORY.csv', 'conversiones.csv', 'sampleSubmission.csv', 'pageviews_complemento.csv', 'SITE_ID.csv', 'CONTENT_CATEGORY_BOTTOM.csv', 'CONTENT_CATEGORY_TOP.csv', 'README.md', 'pageviews.csv']


In [2]:
data = pd.concat([
       pd.read_csv("./data/pageviews.csv", parse_dates=["FEC_EVENT"]),
       pd.read_csv("./data/pageviews_complemento.csv", parse_dates=["FEC_EVENT"])
], sort=False)

In [3]:
data.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
0,2018-03-30 07:35:48,1,1,1,1,1,1,0
1,2018-03-30 07:35:52,2,2,2,2,2,1,0
2,2018-03-30 07:36:11,3,2,2,2,3,1,0
3,2018-03-30 07:36:16,4,2,2,2,3,1,0
4,2018-03-30 07:41:38,5,2,2,2,2,1,0


### Benchmark Kernel

In [4]:
X_test = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test = pd.concat(X_test, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [5]:
X_test.head(5)

,PAGE_1,PAGE_2,PAGE_3,PAGE_4,PAGE_5,PAGE_6,PAGE_7,PAGE_8,PAGE_9,PAGE_10,...,ON_SITE_SEARCH_TERM_285,ON_SITE_SEARCH_TERM_286,ON_SITE_SEARCH_TERM_287,ON_SITE_SEARCH_TERM_288,ON_SITE_SEARCH_TERM_289,ON_SITE_SEARCH_TERM_290,ON_SITE_SEARCH_TERM_291,ON_SITE_SEARCH_TERM_292,ON_SITE_SEARCH_TERM_293,ON_SITE_SEARCH_TERM_295
USER_ID,,,,,,,,,,,,,,,,,,,,,
0,0.028300,0.128440,0.064842,0.043384,0.016949,0.005287,0.000155,0.000155,0.044472,0.114601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.103742,0.157271,0.065846,0.013738,0.006158,0.003316,0.000000,0.000000,0.054950,0.128375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.032517,0.168074,0.114443,0.025760,0.026605,0.002956,0.000000,0.000422,0.027872,0.059966,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.005168,0.192765,0.145736,0.034625,0.040827,0.012920,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.010628,0.120451,0.099839,0.008696,0.046699,0.018035,0.000000,0.000322,0.000966,0.176490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
X_test.shape

(11676, 2174)

In [7]:
X_test.iloc[0,0:65].sum() # la suma de las frecuencias debe ser 1

0.9586378479241177

In [8]:
data = data[data.FEC_EVENT.dt.month < 10]
X_train = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train = pd.concat(X_train, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [9]:
X_train.iloc[:,56]

USER_ID
0        0.001221
1        0.001224
2        0.005242
3        0.007722
4        0.020178
           ...   
11671    0.034161
11672    0.000000
11673    0.016129
11674    0.000000
11675    0.000000
Name: PAGE_57, Length: 11529, dtype: float64

In [10]:
X_train.iloc[0,0:65].sum()

0.9723127035830619

In [11]:
features = list(set(X_train.columns).intersection(set(X_test.columns)))
X_train = X_train[features]
X_test = X_test[features]

In [12]:
y_prev = pd.read_csv("./data/conversiones.csv")
y_train = pd.Series(0, index=X_train.index)
idx = set(y_prev[y_prev.mes >= 10].USER_ID.unique()).intersection(set(X_train.index))
y_train.loc[list(idx)] = 1

In [13]:
print(X_train.shape, X_test.shape)

(11529, 2026) (11676, 2026)


In [14]:
from sklearn import model_selection
from lightgbm import LGBMClassifier

fi = [] # objeto vacío para guardar la importancia de las variables de los modelos entrenados
test_probs = [] # objeto vacío para guardar las probabilidades estimadas
i = 0 
for train_idx, valid_idx in model_selection.KFold(n_splits=10, shuffle=True).split(X_train):
    i += 1
    Xt = X_train.iloc[train_idx]
    yt = y_train.loc[X_train.index].iloc[train_idx]

    Xv = X_train.iloc[valid_idx]
    yv = y_train.loc[X_train.index].iloc[valid_idx]

    learner = LGBMClassifier(n_estimators=10000) # lightgbm with max 10.000 trees
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(Xt, yt), (Xv, yv)],verbose=False)
    
    test_probs.append(pd.Series(learner.predict_proba(X_test)[:, -1],
                                index=X_test.index, name="fold_" + str(i)))
    fi.append(pd.Series(learner.feature_importances_ / learner.feature_importances_.sum(), index=Xt.columns))

test_probs = pd.concat(test_probs, axis=1).mean(axis=1) # mean of probabilities
test_probs.index.name="USER_ID"
test_probs.name="SCORE"
test_probs.to_csv("respuesta", header=True, compression="zip")
fi = pd.concat(fi, axis=1).mean(axis=1) # Variables as explanation of model

# Lado B

### Experimental models

#### Train set and Test set

We split the test set, so we use last three months to test (Given that original test set is not given to competitors)

In [15]:
data = data[data.FEC_EVENT.dt.month < 10]
X_test2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_test2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_test2 = pd.concat(X_test2, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [16]:
data = data[(data.FEC_EVENT.dt.month < 7)]
X_train2 = []
for c in data.drop(["USER_ID", "FEC_EVENT"], axis=1).columns:
    print("haciendo", c)
    temp = pd.crosstab(data.USER_ID, data[c])
    temp.columns = [c + "_" + str(v) for v in temp.columns]
    X_train2.append(temp.apply(lambda x: x / x.sum(), axis=1))
X_train2 = pd.concat(X_train2, axis=1)

haciendo PAGE
haciendo CONTENT_CATEGORY
haciendo CONTENT_CATEGORY_TOP
haciendo CONTENT_CATEGORY_BOTTOM
haciendo SITE_ID
haciendo ON_SITE_SEARCH_TERM


In [17]:
y_prev2 = pd.read_csv("./data/conversiones.csv")
y_test2 = pd.Series(0, index=X_test2.index)
idx = set(y_prev2[y_prev2.mes >= 10].USER_ID.unique()).intersection(set(X_test2.index))
y_test2.loc[list(idx)] = 1

In [18]:
y_train2 = pd.Series(0, index=X_train2.index)
idx = set(y_prev2[(y_prev2.mes < 10) & (y_prev2.mes > 6)].USER_ID.unique()).intersection(set(X_train2.index))
y_train2.loc[list(idx)] = 1

In [19]:
print(X_train2.shape, y_train2.shape, X_test2.shape, y_test2.shape)

(11314, 1890) (11314,) (11529, 2026) (11529,)


In [20]:
features = list(set(X_train2.columns).intersection(set(X_test2.columns)))
X_train2 = X_train2[features]
X_test2 = X_test2[features]

In [21]:
X_train2.shape

(11314, 1890)

#### Some feature selection

In [22]:
from sklearn.decomposition import PCA

pca2 = PCA(0.99,random_state=23)

In [23]:
pca2.fit(X_train2)

PCA(copy=True, iterated_power='auto', n_components=0.99, random_state=23,
    svd_solver='auto', tol=0.0, whiten=False)

In [24]:
X_train_pca2 = pd.DataFrame(pca2.transform(X_train2))
X_test_pca2 = pd.DataFrame(pca2.transform(X_test2))

In [25]:
print(X_train_pca2.shape, X_test_pca2.shape)

(11314, 43) (11529, 43)


In [26]:
# For training and validating

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

selected_features = SelectKBest(chi2,k=200)
selected_features.fit(X_train2, y_train2)
X_train_skb2 = pd.DataFrame(selected_features.transform(X_train2))
X_test_skb2 = pd.DataFrame(selected_features.transform(X_test2))

In [27]:
p_values = pd.DataFrame(selected_features.pvalues_.round(5))
p_values.index.name="FEAT_N"
p_values.columns = ["P_VALUES"]
p_values.sort_values(by=['P_VALUES'])

,P_VALUES
FEAT_N,
1239,0.00005
1026,0.00064
41,0.00272
1019,0.00312
1431,0.02693
...,...
1462,0.99974
1215,0.99976
1299,0.99981


In [28]:
df = pd.DataFrame(selected_features.scores_.round(5))
df.index.name="FEAT_N"
df.columns = ["SCORE"]
df.sort_values(by=['SCORE'],ascending=False)

,SCORE
FEAT_N,
1239,16.40521
1026,11.64837
41,8.98474
1019,8.73733
971,4.89520
...,...
1424,0.00000
960,0.00000
956,0.00000


In [29]:
# For testing

selected_features = SelectKBest(chi2,k=200)
selected_features.fit(X_train, y_train)
X_train_skb_compl = pd.DataFrame(selected_features.transform(X_train))
X_test_skb_compl = pd.DataFrame(selected_features.transform(X_test))

### Decision Tree Classifier

In [30]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(random_state=47)

params = {
    'criterion' : ['gini','entropy'],
    'splitter' : ['best','random'],
    'max_depth' : [None,3,4,5,8,10],
    'max_features' : [None, 10, 25]
}

gs = GridSearchCV(dtc, params, scoring='roc_auc', cv=5)

In [31]:
gs2 = gs.fit(X_train2,y_train2)

In [32]:
gs2.best_score_

0.7929365377576045

In [33]:
gs2.best_estimator_

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [34]:
opt_arbol1 = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=47,
            splitter='best')

In [35]:
predictor2 = opt_arbol1.fit(X_train2,y_train2)

In [36]:
prediccion2 = predictor2.predict_proba(X_test2)

In [37]:
from sklearn.metrics import roc_auc_score

scores = roc_auc_score(y_test2,prediccion2[:,1])
scores

0.8068575687787587

In [38]:
# the same with CV = 2

gs2_no_cv = GridSearchCV(dtc,params,scoring='roc_auc',cv=2)
gs2_no_cv.fit(X_train2,y_train2)
pred_no_cv = gs2_no_cv.best_estimator_
predijo = pred_no_cv.predict_proba(X_test2)

In [39]:
scores_no_cv = roc_auc_score(y_test2,predijo[:,1])
scores_no_cv

0.8068575687787587

In [40]:
gs_pca = gs.fit(X_train_pca2,y_train2)

In [41]:
gs_pca.best_score_

0.7178605582963131

In [42]:
predictor_pca = gs_pca.best_estimator_
pred_pca = predictor_pca.predict_proba(X_test_pca2)

In [43]:
scores_pca = roc_auc_score(y_test2,pred_pca[:,1])
scores_pca

0.7383593018050242

In [44]:
gs_skb = gs.fit(X_train_skb2,y_train2)

In [45]:
gs_skb.best_score_

0.7904701654101584

In [46]:
predictor_skb = gs_skb.best_estimator_
predictor_skb

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=3,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [47]:
predic_skb = predictor_skb.predict_proba(X_test_skb2)

In [48]:
scores_skb_tree = roc_auc_score(y_test2,predic_skb[:,1])
scores_skb_tree

0.8234152386314861

#### DTR - Solution

In [49]:
arbol1 = predictor_skb.fit(X_train_skb_compl,y_train)
sol_arbol1 = arbol1.predict_proba(X_test_skb_compl)

In [50]:
resultado_arbol1 = pd.Series(sol_arbol1[:,1])
resultado_arbol1.index.name="USER_ID" # Renombramos las columnas
resultado_arbol1.name="SCORE" # Renombramos las columnas
resultado_arbol1.to_csv("respuesta_arbol_1", header=True, compression="zip")

### DTR 2

In [51]:
# Optimization through GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

pg = [
    {"splitter":["best","random"],
     "max_depth":[None,1,2,3,4,5,7,10], 
     "max_features":[10,25], 
     "max_leaf_nodes":[10,50,100,150,200]}
]

dtc2 = arbol1

opt = GridSearchCV(dtc2, pg, scoring="roc_auc",n_jobs=-1,cv=StratifiedKFold(5))

In [52]:
opt.fit(X_train2, y_train2)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='entropy', max_depth=3,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=47,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid=[{'max_depth': [None, 1, 2, 3, 4, 5, 7, 10]

In [53]:
opt.best_score_

0.6986078770421436

In [54]:
optimised_decision_tree = opt.best_estimator_
optimised_decision_tree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
                       max_features=25, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [55]:
sol = optimised_decision_tree.predict_proba(X_test2)

In [56]:
scores = roc_auc_score(y_test2,sol[:,1])
scores

0.6385312198566504

In [57]:
opt_arbol2 = DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=25, max_leaf_nodes=10, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=47, splitter='best')

In [58]:
opt_pca = opt.fit(X_train_pca2,y_train2)

In [59]:
opt_pca.best_score_

0.7157811656206726

In [60]:
pca_tree_2 = opt_pca.best_estimator_

In [61]:
res_tree_2 = pca_tree_2.predict_proba(X_test_pca2)

In [62]:
scores = roc_auc_score(y_test2,res_tree_2[:,1])
scores

0.711334591758087

In [63]:
opt_skb = opt.fit(X_train_skb2,y_train2)

In [64]:
opt_skb.best_score_

0.7835440680555065

In [65]:
skb_tree_2 = opt_skb.best_estimator_

In [66]:
sol_skb_2 = skb_tree_2.predict_proba(X_test_skb2)

In [67]:
scores = roc_auc_score(y_test2,sol_skb_2[:,1])
scores

0.8165546699404171

#### DTR2 - solution

In [68]:
arbol2 = skb_tree_2
skb_tree_2.fit(X_train_skb_compl,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
                       max_features=25, max_leaf_nodes=10,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=47, splitter='best')

In [69]:
sol_a2 = skb_tree_2.predict_proba(X_test_skb_compl)
resultado_arbol2 = pd.Series(sol_a2[:,1])
resultado_arbol2.index.name="USER_ID" # Renombramos las columnas
resultado_arbol2.name="SCORE" # Renombramos las columnas
resultado_arbol2.to_csv("respuesta_arbol_2", header=True, compression="zip")

### ADABOOST

In [70]:
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(n_estimators = 8,random_state=23)
clf.fit(X_train2,y_train2)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=8, random_state=23)

In [71]:
probab = clf.predict_proba(X_test2)

In [72]:
scores = roc_auc_score(y_test2,probab[:,1])
scores

0.8264840335436348

In [73]:
clf_pca = clf.fit(X_train_pca2,y_train2)
probab_pca = clf_pca.predict_proba(X_test_pca2)
scores = roc_auc_score(y_test2,probab_pca[:,1])
scores

0.7574733584710601

In [74]:
clf_skb = clf.fit(X_train_skb2,y_train2)
probab_skb = clf_skb.predict_proba(X_test_skb2)
scores = roc_auc_score(y_test2,probab_skb[:,1])
scores

0.8264840335436348

#### ADABOOST - solution

In [80]:
ada_final = AdaBoostClassifier(n_estimators = 8,random_state=23)
ada_final.fit(X_train_skb_compl,y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=8, random_state=23)

In [81]:
sol_ada_final = ada_final.predict_proba(X_test_skb_compl)
resultado_ada = pd.Series(sol_ada_final[:,1])
resultado_ada.index.name="USER_ID" # Renombramos las columnas
resultado_ada.name="SCORE" # Renombramos las columnas
resultado_ada.to_csv("respuesta_ada", header=True, compression="zip")

## RandomForest()

In [82]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold


param_grid = {
    "n_estimators" : [200,500,1000],
    "max_depth" : [3,4],
    "max_features" : [50,100,200],
}

random_forest = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini')

clasificador = GridSearchCV(random_forest,param_grid,scoring="roc_auc",cv=StratifiedKFold(5))

In [83]:
forest2 = clasificador.fit(X_train2,y_train2)

In [84]:
print(forest2.best_score_,forest2.best_estimator_)

0.83349526827768 RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features=100,
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=1000, n_jobs=None, oob_score=False,
                       random_state=47, verbose=0, warm_start=False)


In [85]:
clf_for2 = forest2.best_estimator_

In [86]:
sol_rf2 = clf_for2.predict_proba(X_test2)
scores = roc_auc_score(y_test2,sol_rf2[:,1])
scores

0.8565312706243972

In [87]:
rf_opt = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_features=100,max_depth=4)
rf_opt_pca = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)
rf_opt_skb = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)

In [88]:
rf_opt_pca.fit(X_train_pca2,y_train2)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=47, verbose=0, warm_start=False)

In [89]:
sol_rf22 = rf_opt_pca.predict_proba(X_test_pca2)
scores22 = roc_auc_score(y_test2,sol_rf22[:,1])
scores22

0.8060365158556902

In [90]:
rf_opt_skb.fit(X_train_skb2,y_train2)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=47, verbose=0, warm_start=False)

In [91]:
sol_rf_skb = rf_opt_skb.predict_proba(X_test_skb2)
scores = roc_auc_score(y_test2,sol_rf_skb[:,1])
scores

0.8553940730963249

#### RF - solution

In [92]:
from sklearn.ensemble import RandomForestClassifier

rf_final = RandomForestClassifier(random_state=47,class_weight="balanced",criterion='gini',n_estimators=200,max_depth=4)
rf_final.fit(X_train_skb_compl,y_train)

RandomForestClassifier(bootstrap=True, class_weight='balanced',
                       criterion='gini', max_depth=4, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=200, n_jobs=None, oob_score=False,
                       random_state=47, verbose=0, warm_start=False)

In [93]:
rf_final_sol = rf_final.predict_proba(X_test_skb_compl)
resultado_random = pd.Series(rf_final_sol[:,1])
resultado_random.index.name="USER_ID" # Renombramos las columnas
resultado_random.name="SCORE" # Renombramos las columnas
resultado_random.to_csv("respuesta_random", header=True, compression="zip")


## CATBOOST

In [94]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

In [95]:
cat = CatBoostClassifier(random_seed=47,verbose=0)

grid_cat = {
    "iterations": [50,100,200],
    "max_depth": [None,2,4,6],
}

cat_clf = GridSearchCV(cat,grid_cat,cv=StratifiedKFold(5))

In [96]:
cat_clf.fit(X_train2,y_train2)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             error_score='raise-deprecating',
             estimator=<catboost.core.CatBoostClassifier object at 0x7f76b89e9f10>,
             iid='warn', n_jobs=None,
             param_grid={'iterations': [50, 100, 200],
                         'max_depth': [None, 2, 4, 6]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [97]:
cat_clf.best_estimator_

In [98]:
mod_cat2 = cat_clf.best_estimator_
sol_cat2 = mod_cat2.predict_proba(X_test2)
sco = roc_auc_score(y_test2, sol_cat2[:,1])
sco

0.848346356029593

In [99]:
from sklearn.metrics import roc_auc_score

mod_cat_nogs = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None,verbose=0)
mod_cat_nogs.fit(X_train2,y_train2)
sol_cat_nogs = mod_cat_nogs.predict_proba(X_test2)
sco_cat_nogs = roc_auc_score(y_test2,sol_cat_nogs[:,1])
sco_cat_nogs

0.8466754048727807

In [100]:
mod_cat_skb = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None,verbose=0)
mod_cat_skb.fit(X_train_skb2,y_train2)
sol_cat_skb = mod_cat_skb.predict_proba(X_test_skb2)
sco_cat_skb = roc_auc_score(y_test2,sol_cat_skb[:,1])
sco_cat_skb

0.8512671168073548

#### CATBOOST - solution

In [102]:
cat_final = CatBoostClassifier(iterations=220,random_seed=47,max_depth=None,verbose=0)
cat_final.fit(X_train_skb_compl,y_train)

In [103]:
sol_cat = cat_final.predict_proba(X_test_skb_compl)
resultado_cat = pd.Series(sol_cat[:,1])
resultado_cat.index.name="USER_ID" # Renombramos las columnas
resultado_cat.name="SCORE" # Renombramos las columnas
resultado_cat.to_csv("respuesta_cat", header=True, compression="zip")

### XGBoosting

In [104]:
from xgboost import XGBClassifier

xgb_clf = XGBClassifier(verbosity=0,max_depth=2,sub_sample=0.5)

In [105]:
xgb_clf.fit(X_train_skb2, y_train2)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, sub_sample=0.5, subsample=1, verbosity=0)

In [106]:
sol_xgb = xgb_clf.predict_proba(X_test_skb2)
sco_xgb = roc_auc_score(y_test2,sol_xgb[:,1])
sco_xgb

0.8535749724238829

#### Modelo Final

In [107]:
xgb_final = XGBClassifier(verbosity=0,max_depth=2,sub_sample=0.5)
xgb_final.fit(X_train_skb_compl,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, sub_sample=0.5, subsample=1, verbosity=0)

In [108]:
sol_xgb = xgb_final.predict_proba(X_test_skb_compl)
resultado_xgb = pd.Series(sol_xgb[:,1])
resultado_xgb.index.name="USER_ID" # Renombramos las columnas
resultado_xgb.name="SCORE" # Renombramos las columnas
resultado_xgb.to_csv("respuesta_xgb", header=True, compression="zip")

## Support Vector Machine

In [109]:
from sklearn.svm import SVC

svm = SVC(class_weight="balanced",probability=True,verbose=1,random_state=99,gamma="auto")

In [110]:
svm.fit(X_train_skb2,y_train2)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=True, random_state=99, shrinking=True, tol=0.001,
    verbose=1)

In [111]:
svm_prueba = svm.predict_proba(X_test_skb2)
scoring_svm = roc_auc_score(y_test2, svm_prueba[:,1])
scoring_svm

0.6048455506685189

## Clustering

In [112]:
from sklearn.cluster import KMeans 

kmeans = KMeans(n_clusters=2,random_state=44,verbose=0)
X_train_km = kmeans.fit_transform(X_train_skb2)
y_kmeans = kmeans.predict(X_test_skb2)

In [113]:
scoring = roc_auc_score(y_test2,y_kmeans)
scoring

0.5032292902207474

# STACKING SOLUTION

In [114]:
from lightgbm import LGBMClassifier

models = [
    xgb_final,
    cat_final,
    ada_final,
    rf_final,
    LGBMClassifier()
]

In [115]:
from vecstack import stacking

S_train, S_test = stacking(models,                                             # list of models
                               X_train_skb_compl, y_train, X_test_skb_compl,   # data
                               regression=False,                               # classification task (if you need 
                                                                               #     regression - set to True)
                               mode='oof_pred',                                # mode: oof for train set, fit on full 
                                                                               #     train and predict test set once
                               needs_proba=True,                               # predict probabilities (if you need 
                                                                               #     class labels - set to False) 
                               save_dir='.',                                   # save result and log in current dir 
                                                                               #     (to disable saving - set to None)
                               n_folds=5,                                      # number of folds
                               stratified=True,                                # stratified split for folds
                               shuffle=True,                                   # shuffle the data
                               random_state=0,                                 # ensure reproducibility
                               verbose=1) 

task:         [classification]
n_classes:    [2]
metric:       [log_loss]
mode:         [oof_pred]
n_models:     [5]

model  0:     [XGBClassifier]
    ----
    MEAN:     [0.11679379] + [0.00297508]
    FULL:     [0.11679388]

    Fitting on full train set...

model  1:     [CatBoostClassifier]
    ----
    MEAN:     [0.12067422] + [0.00363037]
    FULL:     [0.12067456]

    Fitting on full train set...

model  2:     [AdaBoostClassifier]
    ----
    MEAN:     [0.45627183] + [0.00659728]
    FULL:     [0.45627252]

    Fitting on full train set...

model  3:     [RandomForestClassifier]
    ----
    MEAN:     [0.42819883] + [0.00452144]
    FULL:     [0.42819862]

    Fitting on full train set...

model  4:     [LGBMClassifier]
    ----
    MEAN:     [0.14252408] + [0.00788062]
    FULL:     [0.14252473]

    Fitting on full train set...

Result was saved to [./[2019.11.23].[19.06.55].075918.ef1c1a.npy]


In [116]:
model_of_models = xgb_final
model_of_models.fit(S_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=2,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, sub_sample=0.5, subsample=1, verbosity=0)

In [121]:
sol_final = model_of_models.predict_proba(S_test)

In [122]:
resultado_final = pd.Series(sol_final[:,1])
resultado_final.index.name="USER_ID" # Renombramos las columnas
resultado_final.name="SCORE" # Renombramos las columnas
resultado_final.to_csv("respuesta_final", header=True)